In [6]:
from pyspark.sql import *
from pyspark.sql.functions import *

spark = SparkSession.builder.getOrCreate() # on crée la session

data = spark.read.csv("data/car.csv", header=True, inferSchema=True, sep=';') # on récupère le csv et on définit les paramètres

data.printSchema() # on affiche le schema

root
 |-- lib_mrq: string (nullable = true)
 |-- lib_mod_doss: string (nullable = true)
 |-- lib_mod: string (nullable = true)
 |-- dscom: string (nullable = true)
 |-- cnit: string (nullable = true)
 |-- tvv: string (nullable = true)
 |-- cod_cbr: string (nullable = true)
 |-- hybride: string (nullable = true)
 |-- puiss_admin_98: integer (nullable = true)
 |-- puiss_max: string (nullable = true)
 |-- typ_boite_nb_rapp: string (nullable = true)
 |-- conso_urb: string (nullable = true)
 |-- conso_exurb: string (nullable = true)
 |-- conso_mixte: string (nullable = true)
 |-- co2: integer (nullable = true)
 |-- co_typ_1: string (nullable = true)
 |-- hc: string (nullable = true)
 |-- nox: string (nullable = true)
 |-- hcnox: string (nullable = true)
 |-- ptcl: string (nullable = true)
 |-- masse_ordma_min: integer (nullable = true)
 |-- masse_ordma_max: integer (nullable = true)
 |-- champ_v9: string (nullable = true)
 |-- date_maj: string (nullable = true)
 |-- Carrosserie: string (nul

In [7]:
data.createOrReplaceTempView("car") # on crée une vue temporaire en la nommant
request = spark.sql("select * from car")
request.show()  #

+----------+------------+---------+--------------------+---------------+-----------+-------+-------+--------------+---------+-----------------+-----------+-----------+-----------+---+-----------+-----------+-----------+-----------+-----+---------------+---------------+--------------------+--------+-----------+---------+----+----+----+----+
|   lib_mrq|lib_mod_doss|  lib_mod|               dscom|           cnit|        tvv|cod_cbr|hybride|puiss_admin_98|puiss_max|typ_boite_nb_rapp|  conso_urb|conso_exurb|conso_mixte|co2|   co_typ_1|         hc|        nox|      hcnox| ptcl|masse_ordma_min|masse_ordma_max|            champ_v9|date_maj|Carrosserie|    gamme|_c26|_c27|_c28|_c29|
+----------+------------+---------+--------------------+---------------+-----------+-------+-------+--------------+---------+-----------------+-----------+-----------+-----------+---+-----------+-----------+-----------+-----------+-----+---------------+---------------+--------------------+--------+-----------+-----

In [8]:
table = spark.table('car')
table.show()

+----------+------------+---------+--------------------+---------------+-----------+-------+-------+--------------+---------+-----------------+-----------+-----------+-----------+---+-----------+-----------+-----------+-----------+-----+---------------+---------------+--------------------+--------+-----------+---------+----+----+----+----+
|   lib_mrq|lib_mod_doss|  lib_mod|               dscom|           cnit|        tvv|cod_cbr|hybride|puiss_admin_98|puiss_max|typ_boite_nb_rapp|  conso_urb|conso_exurb|conso_mixte|co2|   co_typ_1|         hc|        nox|      hcnox| ptcl|masse_ordma_min|masse_ordma_max|            champ_v9|date_maj|Carrosserie|    gamme|_c26|_c27|_c28|_c29|
+----------+------------+---------+--------------------+---------------+-----------+-------+-------+--------------+---------+-----------------+-----------+-----------+-----------+---+-----------+-----------+-----------+-----------+-----+---------------+---------------+--------------------+--------+-----------+-----

### Exercice 1: Calculer la moyenne de la puissance administrative (puiss_admin_98) pour chaque marque de voiture (lib_mrq).



In [20]:
table.select('*') \
                .groupby('lib_mrq') \
                .agg( \
                     round(avg(col('puiss_admin_98')), 2) \
                     .alias('AVG_PUISS') \
                    ) \
                .sort(col('AVG_PUISS').desc()) \
                .show()

+------------+---------+
|     lib_mrq|AVG_PUISS|
+------------+---------+
| LAMBORGHINI|    53.39|
|     FERRARI|    52.44|
|     BENTLEY|    48.15|
| ROLLS-ROYCE|    44.91|
|ASTON MARTIN|    38.41|
|    MASERATI|    29.08|
|     PORSCHE|    28.46|
|      JAGUAR|    26.67|
|    CADILLAC|    23.37|
|  LAND ROVER|     21.7|
|       LOTUS|    21.14|
|       LEXUS|    18.83|
|    INFINITI|    17.69|
|        JEEP|    16.43|
|   CHEVROLET|    14.43|
|         BMW|    14.36|
|        AUDI|    12.92|
|    MERCEDES|    10.99|
|       VOLVO|    10.81|
|      SUBARU|    10.26|
+------------+---------+
only showing top 20 rows



### Exercice 2: Trouver la voiture ayant la consommation mixte (conso_mixte) la plus basse.



In [43]:
table.select('lib_mrq','lib_mod_doss','lib_mod','conso_mixte').filter(col('conso_mixte') == table.select('*').agg(min(col('conso_mixte'))).collect()[0][0]).show()

+-------+------------+-------+-----------+
|lib_mrq|lib_mod_doss|lib_mod|conso_mixte|
+-------+------------+-------+-----------+
|    BMW|     SERIE I|     I3|0,600000024|
+-------+------------+-------+-----------+



### Exercice 3: Filtrer les voitures dont la puissance maximale (puiss_max) est supérieure à 200 chevaux.



In [48]:
table.filter(col('puiss_max') > 200).show()

+------------+--------------------+---------+--------------------+---------------+------------------+-------+-------+--------------+---------+-----------------+-----------+-----------+-----------+---+-----------+-----------+-----------+-----+----+---------------+---------------+--------------------+--------+-----------+-----+----+----+----+----+
|     lib_mrq|        lib_mod_doss|  lib_mod|               dscom|           cnit|               tvv|cod_cbr|hybride|puiss_admin_98|puiss_max|typ_boite_nb_rapp|  conso_urb|conso_exurb|conso_mixte|co2|   co_typ_1|         hc|        nox|hcnox|ptcl|masse_ordma_min|masse_ordma_max|            champ_v9|date_maj|Carrosserie|gamme|_c26|_c27|_c28|_c29|
+------------+--------------------+---------+--------------------+---------------+------------------+-------+-------+--------------+---------+-----------------+-----------+-----------+-----------+---+-----------+-----------+-----------+-----+----+---------------+---------------+--------------------+----

### Exercice 4: Calculer le nombre de voitures hybrides par carrosserie.



In [ ]:
table.select('Carrosserie').groupby(col('Carosserie')).filter(col('hybride') == 'oui').show()

### Exercice 5: Calculer le pourcentage de voitures émettant moins de 120 g/km de CO2 (co2) par gamme de voiture.



### Exercice 6: Trouver la carrosserie la plus courante parmi les voitures de chaque gamme.



### Exercice 7: Calculer la différence entre la puissance maximale (puiss_max) et la puissance administrative (puiss_admin_98) pour chaque voiture.



### Exercice 8: Trouver les marques de voiture (lib_mrq) dont la consommation urbaine (conso_urb) est inférieure à la consommation mixte (conso_mixte).



### Exercice 9: Calculer la moyenne des émissions de CO2 (co2) par année de mise à jour (annee_maj).



### Exercice 10: Trouver les voitures dont la masse ordma_max est supérieure à la moyenne de la masse ordma_max de toutes les voitures.